# Information extraction

**Previously**:

* Text sources
* Information retrieval (text search)

Both information retrieval and information extraction aim to respond to _information needs_ on the basis of a (typically large) collection of texts. However, there are notable differences between the approaches (prototypical case):

| <font size="4">Information retrieval</font> | <font size="4">Information extraction</font> |
| ----------- | ----------- |
| <font size="3">Ad-hoc information needs</font> | <font size="3">Persistent information needs</font> |
| <font size="3">Response using "live" database query</font> | <font size="3">Response using dedicated IE system</font> |
| <font size="3">Unstructured output (documents)</font> | <font size="3">Structured output (e.g. relations)</font> |
| <font size="3">Results discarded after query</font> | <font size="3">Results stored in database after extraction</font> |
| <font size="3">Example: <a href="https://google.com">Google</a></font> | <font size="3">Example: <a href="https://string-db.org/">STRING DB</a></font> |


---

<center>(<b>NOTE: the following is a recap of material presented in the Introduction to Language Technology course.</b> If you've recently taken this course or are otherwise familiar with NER concepts, feel free to skip this and move to the material on NER using BERT.)</center>

---

# Named Entity Recognition

* Recognize mentions of *named entities* (people, places, companies, etc.) and their types (`PER[SON]`, `LOC[ATION]`, etc.) in text
    * Commonly expanded to include also mentions of e.g. dates
    * In specialized domains, recognize names of e.g. genes, proteins, and chemicals
* Basic task in information extraction: need to know what things are talked about in order to extract e.g. relations between them
* Reliable for well-studied text domains in resource-rich languages (e.g. English news), often well over 90% precision and recall
    * Much worse for e.g. Finnish Twitter

<img src="https://raw.githubusercontent.com/TurkuNLP/Text_Mining_Course/master/figs/english_ner_example.png">

---

<img src="https://raw.githubusercontent.com/TurkuNLP/Text_Mining_Course/master/figs/finnish_ner_example.png" width="90%" align="left">

### Approaches

* **Dictionary lookup**: simple and fast to implement and apply, but no list is complete, no answer to ambiguity
    * e.g. "Nokia" as `ORG` vs `LOC`, "Bush" as `PER` vs common noun 
* **Rule-based systems**: able to take context into account, but require manual tuning, can be brittle
* **Machine learning**: state-of-the-art results, can be retrained, but supervised ML requires manual annotation

# Sequence labeling

* Fundamental machine learning task: assign a class (label) to each item in a sequence
    * Given observed input sequence ***x*** = { *x<sub>1</sub>*, *x<sub>2</sub>*, ... *x<sub>n</sub>* }, predict outputs ***y*** = { *y<sub>1</sub>*, *y<sub>2</sub>*, ... *y<sub>n</sub>* }
* Examples in NLP: part-of-speech tagging, chunking, **named entity recognition**, speech recognition
    * e.g. for part-of-speech tagging, the input ***x*** is a sequence of words and the output ***y*** a sequence of POS tags

<hr>
    
```
x:    The  quick  brown  fox   jumps  over  the  lazy  dog
       ↓     ↓      ↓     ↓     ↓      ↓     ↓    ↓     ↓
y:    DET   ADJ    ADJ   NOUN  VERB   ADP   DET  ADJ   NOUN
```

<hr>

* Simply predicting ***x*** ↦ *y<sub>i</sub>* separately for each *y<sub>i</sub>* misses dependencies between outputs 
    * For example, in English, `ADJ` and `DET` are likely before `NOUN` but unlikely after

## NER as sequence labeling

* Given a sequence of words, determine for each whether it is (a part of) a name
* *In-Out-Begin* (IOB) encoding (or BIO):
    * A token can <b>B</b>egin an entity mention, be <b>I</b>nside one, or be <b>O</b>utside of mentions
    * Labels also encode entity type: **B-PER**, **I-PER**, **B-ORG**, **I-ORG**
* NER as multiclass classification: assign each word to one class

<hr>

```
x:    Adams  will  miss  England's   opening  World   Cup  qualifier
        ↓     ↓     ↓       ↓           ↓       ↓      ↓      ↓
y:    B-PER   O     O     B-LOC         O     B-MISC I-MISC   O
```

<hr>

* Simple representation applicable with most machine learning methods
* Limitation: discontinuous and overlapping (e.g. nested) spans

# Learning sequence labeling

* Supervised machine learning: requires annotated data for training
    * We assume examples of inputs ***x*** with correct outputs ***y***
    * Train ML method to predict outputs for new, unseen inputs → Need to *generalize*, not simply memorize (x,y) pairs from training data
* Key question: how to represent the words to the ML method as *features*
    * Examples: words (``Finland``) prefixes/suffixes (`Fin-`, `-and`), POS tags (`NOUN`), word shape (`STARTS-WITH-CAPITAL`), ...
    * **Deep learning approach**: feature learning, often combining unsupervised pre-training with task-specific fine-tuning


## Naive Bayes

* Simple probabilistic classification method popularized by success in spam filtering
* Predict probability each class <i>y</i> given input <i>x</i> using Bayes' rule $P(y|x) = \frac{P(y)P(x|y)}{P(x)}$, pick most likely 
* Estimate probabilities based on (x,y) counts in training data (+smoothing), e.g.
    * Prior: `P(B-PER) = count(*, B-PER) / total-examples`
    * Conditional: `P(Adam|B-PER) = count(Adam, B-PER) / count(*, B-PER)`
* Simple and fast, but limited: assumes independence of features (the "naive" part -- almost always false!) and independence of labels (no notion of sequence)

## Hidden Markov Models (HMM)

* "Sequence version" of Naive Bayes: in addition to output probabilities, model also *transition probabilities* $P(y_i|y_{i-1})$
    * Markov assumption: probability of state depends only on previous state (not whole history)

<img src="https://raw.githubusercontent.com/TurkuNLP/Text_Mining_Course/master/figs/bayes_hmm.png" width="50%">

<div style="text-align:center; color:gray; font-size:80%">(Figure modified from <a href="https://arxiv.org/pdf/1011.4088v1.pdf">Sutton and McCallum (2011)</a>)</div>

* Assume an underlying "hidden" sequence of labels, which generates the observed data

<hr>

```
y:    DET →  ADJ  →  ADJ →  NOUN → VERB → ADP → DET → ADJ → NOUN
       ↓      ↓       ↓      ↓      ↓      ↓     ↓     ↓     ↓
x:    The   quick   brown   fox   jumps   over  the  lazy   dog
```

<hr>

* Transition probabilities can also be straightforwardly estimated from data, e.g.
    * `P(I-PER|B-PER) = count-seq(I-PER, B-PER) / count-seq(*, B-PER)`
* Decoding: [Viterbi algorithm](https://en.wikipedia.org/wiki/Viterbi_algorithm) - efficient polynomial algorithm to find the best hidden sequence of labels for the observed data
* Incorporates sequence information, but otherwise retains the independence assumptions from Naive Bayes


## Conditional Random Fields (CRF)

(Specifically, linear-chain CRFs)

* *Discriminative* sequence classifier: estimate conditional probability $P(\textbf{y}|\textbf{x})$ (this is what we're interested in)
    * We know the observed data (and don't care about it's probability) and want to predict the output (i.e. discriminate between possible outputs)
    * (By contrast, *generative* models such as NB and HMM estimate the joint distribution $P(\textbf{x},\textbf{y})$)

<img src="https://raw.githubusercontent.com/TurkuNLP/Text_Mining_Course/master/figs/sutton_mccallum_models.png" width="80%">

<div style="text-align:center; color:gray; font-size:80%">(Figure modified from <a href="https://arxiv.org/pdf/1011.4088v1.pdf">Sutton and McCallum (2011)</a>)</div>

* Avoid the independence assumptions made by HMMs (HMM is a special case of CRF)
* The go-to ML method for many sequence classification tasks since its introduction in 2001 ([paper](https://people.cs.umass.edu/~mccallum/papers/crf-icml01.ps))
* Trained iteratively using gradient descent; can get stuck in a local optimum and can be slow
* Decoded in much the same way as HMMs - efficient polynomial algorithm to find the best sequence of labels

We won't go into any real details here, you can check out one of the many tutorials out there if you want to know more about the inner workings of CRFs and the way they're trained (e.g. [this one](http://www.cs.upc.edu/~aquattoni/AllMyPapers/crf_tutorial_talk.pdf)). For a comprehensive introduction, look <a href="https://arxiv.org/pdf/1011.4088v1.pdf">here</a>.

---

### Implementations

* [CRFsuite](http://www.chokkan.org/software/crfsuite/) is a good general CRF training software
* [sklearn-crfsuite](https://sklearn-crfsuite.readthedocs.io/en/latest/) CRFsuite Python wrapper
* [NERSuite](http://nersuite.nlplab.org/) a driver script for *CRFsuite* with predefined features tuned for the NER task
* [CoreNLP](https://stanfordnlp.github.io/CoreNLP/) also has a NER annotator (remember we played with it on the first lectures)

## Deep learning approaches

* State-of-the-art in NER as in many other machine learning tasks
* Recurrent neural networks (RNNs) with e.g. LSTM cells previously a popular choice
* Recently increasing focus on Transformer-based models such as BERT
* (Previous work not obsolete: e.g. CRF losses still popular with Transformers for NER)

We'll be studying a BERT-based approach to NER in detail shortly.

<a href="https://raw.githubusercontent.com/TurkuNLP/Text_Mining_Course/master/figs/bert-for-token-classification.png"><img src="https://raw.githubusercontent.com/TurkuNLP/Text_Mining_Course/master/figs/bert-for-token-classification.png" width="400px"/></a>

<div style="color:gray; font-size:80%">(Figure adapted from <a href="http://jalammar.github.io/illustrated-bert/">The Illustrated BERT, ELMo, and co.</a> CC BY-NC-SA)</div>

# Evaluation

Sequence tagging methods are typically evaluated using the following metrics:

* precision: fraction of predicted items that were correct
* recall: fraction of correct items that were predicted
* F1-score: harmonic mean of precision and recall

These metrics are normally calculated for _mentions_ rather than words: instead of looking at whether individual words are tagged correctly or incorrectly, the evaluation determines whether named entity mentions, potentially consisting of multiple words, are correctly recognized.

This means that a correctly tagged multi-word mention such as `Time Warner Cable` counts as one correct name, not three correct tags. (No partial credit is given for e.g. `Time Warner`.) For datasets with many multi-word mentions, this is a considerably more demanding metric.

(The `pip`-installable library [`seqeval`](https://pypi.org/project/seqeval/) is one Python implementation of standard NER metrics.)

#  Training data for NER

Supervised machine learning methods require annotated data for training. Datasets available for various languages and domains, e.g.:

* https://www.clips.uantwerpen.be/conll2002/ner/: Spanish, Dutch
* https://www.clips.uantwerpen.be/conll2003/ner/: English, German
* https://github.com/turkunlp/turku-ner-corpus: Finnish
* http://biocreative.org: English (biomedical)
* https://catalog.ldc.upenn.edu/LDC2013T19: English, Mandarin Chinese, Arabic, Chinese

**Next**: named entity recognition with BERT